In [1]:
from Collection.Financial import INVESTING as iv
import psycopg2 
import yfinance as yf
import pandas as pd
from tqdm import tqdm

In [2]:
iv.information()


        라이브러리는 2개로 나누어집니다. 

        데이터를 수집하는 라이브러리인 Investing_Crawler, 데이터를 가공하는 라이브러리인 Investing_Cleanse 

        ------------------------------------------------------------------------------------------

        Investing_Crawler의 함수는 아래와 같습니다. 

        DriverSettings()은 셀레니움 크롬 드라이버 세팅 함수입니다. 

        download_historial()은 과거 주종가 데이터를 수집하는 함수입니다. 

        invesingDotcom_crawler()은 인베스팅 닷컴에서 데이터를 수집하는 함수입니다. 

        ------------------------------------------------------------------------------------------

        Investing_Cleanse는 클래스를 실행시키면 바로 진행이 됩니다. 

        


In [3]:
crawler = iv.Investing_Crawler("investingDotcomColumns.xlsx")

In [4]:
crawler.DriverSettings()

In [5]:
crawler.investingDotcom_crawler("mexico", "mexico", "/")

AttributeError: 'NoneType' object has no attribute 'find_all'